In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
import tensorflow as tf

In [ ]:
# 분노 불러오기
sen = pd.read_csv('/Users/dawon/Documents/최종_전처리/분노.csv')
sen.columns = ['index','a']     # 컬럼명 정리
sen = sen.loc[sen['a'].notnull(),:]   # null 처리
only_word = list(sen['a'].str.split())  # 리스트형태로 만들기
rage = np.repeat('2',len(only_word))


# 긍정 불러오기
sen2 = pd.read_csv('/Users/dawon/Documents/최종_전처리/긍정.csv')
sen2.columns = ['index','a']    # 컬럼명 정리
sen2 = sen2.loc[sen2['a'].notnull(),:]   # null 처리
only_word2 = list(sen2['a'].str.split())    # 리스트형태로 만들기
positive = np.repeat('1',len(only_word2))

# 중립 불러오기 

sen3 = pd.read_csv('/Users/dawon/Documents/최종_전처리/중립.csv')
sen3.columns = ['index','a']    # 컬럼명 정리
sen3 = sen3.loc[sen3['a'].notnull(),:]   # null 처리
only_word3 = list(sen3['a'].str.split())    # 리스트형태로 만들기
neutrality = np.repeat('0',len(only_word3))

In [ ]:
# 감정 별 train, test 분리(y는 필요하지 않음)
df1 = pd.DataFrame({'text':only_word, 'tag' : rage}).reset_index()
df2 = pd.DataFrame({'text':only_word2, 'tag' : positive}).reset_index()
df3 = pd.DataFrame({'text':only_word3, 'tag' : neutrality}).reset_index()

train_x1, test_x1, train_y1, test_y1 = train_test_split(df1.iloc[:,1:], df1.iloc[:,0], random_state = 0, train_size = 0.8) 
train_x2, test_x2, train_y2, test_y2 = train_test_split(df2.iloc[:,1:], df2.iloc[:,0], random_state = 0, train_size = 0.8) 
train_x3, test_x3, train_y3, test_y3 = train_test_split(df3.iloc[:,1:], df3.iloc[:,0], random_state = 0, train_size = 0.8) 

In [ ]:
df1['text']

0           [외국인, 입국, 안, 막, 검사, 다, 나오, 노벨, 평화상, 타, 이러, 아니]
1                                           [ㅋ, 인간, 극장, 이]
2        [성, 석문, 숙, 가게, 드나들, 경찰, 대해, 직무, 유기, 고발장, 냈, 대구...
3          [전, 병원, 세균, 미세먼지, 주범, 인, 짱깨, 먼저, 없애, 제일, 중요, 하]
4                         [중국, 엄청, 많, 농심, 삼양, 라면, 인, 알, 샀]
                               ...                        
11204    [국민, 세금, 국민, 관계, 없, 사람, 지원, 정책, 당국, 마음대로, 써도, ...
11205               [걸려, 이, 청와대, 시키, 보내, 똘, 아이, 조심, 해야, 맞]
11206    [짱깨, 그렇, 뭐, 공산주의, 국가, 든, 조작, 가능, 공산주의자, 문재인, 하...
11207               [논란, 될, 알, 왜, 추가, 검사, 안, 했, 왜, 나쁜, 인간]
11208    [일본, 확진, 추세, 보, 장난질, 치, 이, 쉽, 알, 있, 어차피, 병, 걸리...
Name: text, Length: 11209, dtype: object

In [ ]:
# 모든 감정 합치기 
df1 = pd.concat([train_x1, train_x2, train_x3])
df2 = pd.concat([test_x1, test_x2, test_x3])

In [ ]:
sentences=df1['text'].apply(lambda x : ' '.join(x)).to_list()
sentences

['작년 채 작년 혐의 떴 뭐 이번 없 죄 뒤집 씌우',
 '사회 거리 두기 단계 한다고 그런다고 하 참',
 '아 요즘 북한 챙길 이해 안 가 대체 북한 왜 챙겨 국내 어수선 자체 인데',
 '김어준 같이 개보 다 넘 지역 감정 계속 부추기 개보 다 김어준',
 '야 미친 돈 해라 세금 쓰',
 '방역 작업 모 ㅋㅋㅋ ㅋㅋ 이미 서울 시내 활보 바이러스 퍼트리 ㅋㅋㅋ ㅋㅋㅋ ㅋㅋㅋ 글러 먹 대한민국',
 '꼴값',
 '무식 그냥 코로나 엑기스 뿌려',
 '대한민국 경쟁 상대 미국 대한민국 박근혜 때 살 좋 나라 불가리아 민요 만주 환상곡 안익태 애국가 주적 쿠데타 세력 국민 힘 로고 루마니아 국기 모방 국민 힘 곡구 민 노 찌 꺼라 일본 극우 표절 반공 국시 빨대',
 '잘 떠올려 개돼지 대한민국 경제 외교 국방 언론 지금 망가진 있',
 '왜 교육 당국 자꾸 개학 등교 재촉 물론 한없이 안 할 없 지금 아니 연휴 고비 라면서 연휴 직후 중 개학',
 '대단 뇌 없',
 '그냥 내버려 요트 쇼핑 하 가 추석날 부모 못 만나 맨날 국민 탓 해',
 '짜 구리 쳐먹 아니 공식 문다 혜 조민 정경 간경 애 미추 박영선 유은혜 에잇 썸 파구 리도 하심',
 '개돼지 국민 맞춤 일자리',
 '정은경 긴말 싫 마음대로 하 국민 옥죄 남불 방역 역겹 외국인 왜 아직 무료 치료 니 돈 받',
 '중국 여행 가 모조리 돼 인천 공항 패 쇄 빨갱이 들',
 '해외 냐 ㅋㅋ 그냥 코로나 잠잠 싫 니 막 그나마 나 왜 이렇게 말 듣 참 한결 같 짜 피 더 늘어나 그때 막 하 지금 막 더 이득 일 생각 못 하',
 '걸려서 가족 걸려서 죽 생각 인권 지금 중요 중단 심각 상황 알리 그냥 다닌다 말 없 살인 이 인권 찾 지금 아니',
 '격리 어기 사람 많 세상 이틀 밖 싸돌아다녔 모르 격리 안 하 관리 되',
 '실드 칠 쳐야지 ㅋㅋ ㅋㅋ 분명 실드 친구 사람 연예인 클럽 갔 때 댓글 욕 오지 박 아이디 대조 실제로 사람 있 ㅋㅋㅋ ㅋㅋ 진짜 팬 이 실드 치 채찍 해서 수 정신 차

In [ ]:
sentences2=df2['text'].apply(lambda x : ' '.join(x)).to_list()
sentences2

['욕 나온다 진짜',
 '선생 미쳤',
 '말 하 문제아 먼저 코로나 걸려 엎어져야 다 그래야 정신 차리',
 '멍청 질병 첨 이 중국 자료 있',
 '다 처먹 문가 늠 좀 처 먹',
 '무조건 민주당 빼 다 정신 챙기',
 'ㅁ 짬 개 그만 알랑방귀 뀌 빨간 노마',
 '좀 돌아다니',
 '그냥 조용히 하 검사 수 수작질 하',
 '세월 세자 만들 화 난다 해상 사고 음모 만들 지랄 이',
 '전광훈 모욕 이 최대한 예우 해 같 라고 불러야 적반하장 따로 없 잘못 사과 모르 뉘우칠 모르 인간 인간 대접 받 없 전광훈 사랑 제일 교회 코로나 확산 전적 인 책임 있 코 나가 방역 당국',
 '정치 휘둘 필요 없',
 '일자리 만든다고 정도 쓴 같 도대체 지금 일자리 어떻게 만들 낱낱 히 공개 국민 세금 한 국민 알 권리 있 취업 잘 된 아니 또 쓴다는 도저히 이해 안 됨 조면 만든다고 하 실업자',
 '진짜 이해 안 된다 수도 그렇게 가 안전 문자 보내 어쩔 없이 갔 해도 요즘 서울 경기 지역 심한 알 증상 있 없 갔 해도 자 격리 사람 많 다 쳐 쑤시 돌아다녔 기사 무증상 아니 증',
 '영상 통화 뒤통수 치 섞 홍어 새',
 '지랄',
 '코로나 세계 퍼트리 사과 없 짱 골라 새끼 웃기 한심 짱 골라 새끼',
 '빨 비상사태 선포 무능 인간',
 '문괴 조문 안 가',
 '지역 감정 조장 국민 아니 극단 주의자 말 일반 싸움 만드 같 기러기 이 속 마음 왜 사람 끌 들 차도살인 자질 없 흠 기사 라고',
 '자 한당 이 견 찰 제대로 기소 안 했 무죄 미리 알 나쁜 검찰 새이',
 '오래 사 극',
 '제정신 이 중국 꺼 이용 사람 없 다',
 '문재인 중국 마스크 상납 안 했어도 학생 저렇 마스크 쪼가리 산다고 서다 목숨 잃 문재인 대신 죽 낫',
 '웃기 소리 하 지나가 개 다 웃 마스크 봅시다 코로나 중국 전멸 때 자국 마스크 미리 준비 국민 안전 지켜야 수장 인 대통령 짜 파 축제 웃 떠들 이제 다 잡혀가 다 경제 살리 국민 심리 무장 해제 시켰 중

# 내가하고 있는거 토크나이저 추가

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(sentences)
vocab_size = len(t.word_index) +1
print(vocab_size)

23043


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(sentences2)
vocab_size2 = len(t.word_index) +1
print(vocab_size2)

11933


In [ ]:
X_encoded = t.texts_to_sequences(sentences)
print(X_encoded)

[[1074, 3578, 1074, 8838, 10799, 57, 146, 5, 393, 1471, 1355], [162, 237, 702, 325, 668, 3653, 2, 83], [70, 268, 99, 3259, 319, 3, 24, 478, 99, 21, 1143, 273, 6167, 335, 50], [2969, 198, 4, 135, 152, 2171, 132, 2791, 4, 2969], [43, 145, 54, 161, 96, 46], [67, 2405, 1680, 17, 16, 191, 255, 9014, 4465, 61, 818, 17, 17, 17, 3296, 45, 66], [], [773, 48, 8, 2518], [66, 6138, 1424, 74, 66, 381, 30, 62, 23, 10, 5294, 6321, 8665, 751, 9, 217, 3607, 9, 217, 860, 704, 1433, 7453, 37, 1227, 11361, 4639], [20, 363, 66, 98, 960, 2266, 158, 34, 7], [21, 595, 623, 796, 302, 413, 933, 3, 100, 5, 34, 14, 1428, 1636, 8794, 1428, 1622, 302], [223, 470, 5], [48, 2074, 1587, 1658, 2, 24, 593, 25, 991, 694, 9, 95, 47], [404, 2558, 1231, 14, 5615, 7944, 2186, 11217, 4149, 7256, 84, 1570, 9506], [363, 9, 9469, 540], [1847, 150, 3650, 2, 9, 10656, 1273, 67, 2715, 251, 21, 144, 1408, 109, 716, 54, 28], [6, 301, 24, 5023, 612, 879, 1049, 886, 3415, 421, 125], [105, 63, 16, 48, 8, 1395, 150, 716, 72, 678, 184, 21

In [ ]:
X_encoded2 = t.texts_to_sequences(sentences2)
print(X_encoded2)

[[187, 768, 27], [939, 731], [13, 2, 5880, 211, 8, 566, 5881, 4, 1011, 107, 402], [940, 539, 1258, 1, 6, 1520, 7], [4, 284, 1521, 1715, 19, 596, 45], [291, 131, 318, 4, 107, 567], [659, 5882, 88, 164, 5883, 4026, 3095, 2545], [19, 520], [48, 1158, 2, 60, 212, 5884, 2], [377, 5885, 90, 691, 358, 5886, 732, 2546, 90, 389, 1], [390, 3096, 1, 2164, 5887, 47, 15, 93, 5888, 4027, 873, 5, 151, 420, 92, 5889, 92, 53, 53, 2165, 28, 5, 390, 359, 298, 75, 8, 264, 3097, 22, 216, 7, 69, 213, 67, 623], [104, 5890, 173, 5], [540, 3098, 94, 817, 15, 292, 34, 540, 147, 90, 5891, 1522, 541, 9, 96, 230, 9, 35, 2166, 7, 1523, 20, 241, 14, 38, 4028, 4029, 319, 3, 441, 5892, 3098, 2, 5893], [27, 319, 3, 200, 874, 122, 24, 460, 1259, 165, 568, 176, 188, 177, 268, 255, 769, 152, 2167, 35, 521, 7, 5, 188, 177, 142, 136, 11, 39, 4, 242, 5894, 1899, 65, 875, 14, 2168], [1524, 1383, 1900, 137, 4030, 1901, 1012], [389], [8, 42, 818, 420, 5, 110, 2547, 138, 624, 299, 110, 2547, 138], [276, 5895, 2169, 285, 53], [58

In [ ]:
max_len = max(len(l) for l in X_encoded)
print(max_len)

50


In [ ]:
max_len2 = max(len(l) for l in X_encoded2)
print(max_len2)

75


In [ ]:
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# X_train = pad_sequences(X_encoded, maxlen = max_len, padding = 'post')

# labels_train = np.array(df1['tag']).reshape(-1,1)
# labels_test = np.array(df2['tag']).reshape(-1,1)
# y_train = np.array(labels_train)
# y_test = np.array(labels_test)

# print(y_train)


In [ ]:
# sum(y_train,[])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(X_encoded, maxlen = max_len2, padding = 'post')
X_test = pad_sequences(X_encoded2, maxlen = max_len2, padding = 'post')
from sklearn.preprocessing import OneHotEncoder
# one hot encoding
labels_train = np.array(df1['tag']).reshape(-1,1)
labels_test = np.array(df2['tag']).reshape(-1,1)
onehotencoder = OneHotEncoder()
onehotencoder.fit(labels_train)
train_y = onehotencoder.transform(labels_train).toarray()
test_y = onehotencoder.transform(labels_test).toarray()

In [ ]:
print(X_train)

[[1074 3578 1074 ...    0    0    0]
 [ 162  237  702 ...    0    0    0]
 [  70  268   99 ...    0    0    0]
 ...
 [ 563 2340  381 ...    0    0    0]
 [ 336 4127 2215 ...    0    0    0]
 [  76 1539  523 ...    0    0    0]]


In [ ]:
print(X_test)

[[ 187  768   27 ...    0    0    0]
 [ 939  731    0 ...    0    0    0]
 [  13    2 5880 ...    0    0    0]
 ...
 [ 105  195   21 ...    0    0    0]
 [1565  918 3952 ...    0    0    0]
 [ 133  189   61 ...    0    0    0]]


In [ ]:
import gensim
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/Users/dawon/Documents/최종_전처리/naver_w2v',binary = False)
print(word2vec_model.vectors.shape)

(10970, 100)


In [ ]:
import numpy as np
import gensim 

#인코딩 이슈 때문에 오류 남 binary = true
# word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/Users/dawon/Documents/최종_전처리/naver_w2v', binary = True)
# print(word2vec_model.vectors.shpe)
embedding_matrix = np.zeros((vocab_size,100)) 
#단어 집합 크기의 행과 10000개의 열을 가지는 행렬 생성, 값은 전부 0으로 채워진다.
print(np.shape(embedding_matrix))

(23043, 100)


In [ ]:
embedding_matrix2 = np.zeros((vocab_size2,100)) 
#단어 집합 크기의 행과 10000개의 열을 가지는 행렬 생성, 값은 전부 0으로 채워진다.
print(np.shape(embedding_matrix2))

(11933, 100)


In [ ]:
word2vec_model

In [ ]:
#word2vec

def get_vector(text):
    if text in word2vec_model:
        return word2vec_model[text]
    else:
        return None
    
for text, i in t.word_index.items():
    temp = get_vector(text)
    if temp is not None:
        embedding_matrix[i] = temp
    

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding, Flatten

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding

In [ ]:
model = Sequential()
e = Embedding(vocab_size, 100, weights = [embedding_matrix], input_length = max_len2, trainable =False)
model.add(e)
model.add(Flatten())
model.add(Dense(3, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics =['acc'])
model.fit(X_train,train_y, epochs=20, batch_size=512)

model.summary()

Epoch 1/20
54/54 [==============================] - 0s 4ms/step - loss: 0.8251 - acc: 0.6288
Epoch 2/20
54/54 [==============================] - 0s 4ms/step - loss: 0.7383 - acc: 0.6579
Epoch 3/20
54/54 [==============================] - 0s 5ms/step - loss: 0.7119 - acc: 0.6670
Epoch 4/20
54/54 [==============================] - 0s 5ms/step - loss: 0.6960 - acc: 0.6748
Epoch 5/20
54/54 [==============================] - 0s 5ms/step - loss: 0.6856 - acc: 0.6787
Epoch 6/20
54/54 [==============================] - 0s 5ms/step - loss: 0.6782 - acc: 0.6828
Epoch 7/20
54/54 [==============================] - 0s 5ms/step - loss: 0.6726 - acc: 0.6840
Epoch 8/20
54/54 [==============================] - 0s 5ms/step - loss: 0.6676 - acc: 0.6869
Epoch 9/20
54/54 [==============================] - 0s 5ms/step - loss: 0.6637 - acc: 0.6888
Epoch 10/20
54/54 [==============================] - 0s 5ms/step - loss: 0.6598 - acc: 0.6920
Epoch 11/20
54/54 [==============================] - 0s 5ms/step - lo

In [ ]:
# test 결과 확인
results = model.evaluate(X_test, test_y, batch_size=512)

14/14 [==============================] - 0s 3ms/step - loss: 0.7106 - acc: 0.6617
